## System implementation - Unmute

Great! Now I can give you a complete picture of how the full Unmute system works.

System Overview

**Unmute** is a cascaded system that lets you have voice conversations with any text-based LLM. It's called "cascaded" because it chains three separate components:

1. **DSM-ASR** (Speech-to-Text) - transcribes what you say
2. **LLM** (any text model) - generates text response
3. **DSM-TTS** (Text-to-Speech) - speaks the response

Architecture

```
User Browser ←→ Backend ←→ STT Server
                         ↓
                         LLM Server
                         ↓
                         TTS Server
```

The user's browser connects via WebSocket to a backend server, which orchestrates everything:

**Flow:**
1. Browser sends audio → Backend → STT transcribes it
2. STT detects end of speech (semantic VAD)
3. Backend sends transcript → LLM generates response
4. As LLM generates tokens → TTS starts speaking immediately (streaming in text!)
5. Audio streams back to browser

Key Innovation: Streaming in Text

Unlike other TTS systems that need the complete text first, DSM-TTS can start generating audio as soon as it receives the first few tokens from the LLM. This dramatically reduces latency.

The system uses a "flush trick" to reduce latency further - when speech ends, it processes remaining audio at 4x speed.

### Frontend

Frontend Technology

The frontend is a **Next.js app** (React-based framework) located in the `frontend/` directory. It runs on port 3000 by default.

Communication Protocol

The frontend and backend communicate via **WebSocket** using a protocol based on the **OpenAI Realtime API** (ORA). However, Unmute makes some modifications:
- Some extra message types were added
- Some parameters are simplified
- Not fully compatible with ORA yet (but they're working toward it)

The protocol details are defined in `unmute/openai_realtime_api_events.py`.

Audio Processing

The browser:
- Captures audio from the user's microphone
- Sends it over WebSocket to the backend in real-time
- Receives audio back from the TTS
- Plays it to the user

#UI Features

**Keyboard shortcuts:**
- Press **S** for subtitles (shows transcription for both user and chatbot)
- Press **D** for dev mode (debug view with extra info)

**User controls:**
- Can interrupt the AI mid-response
- Can change voices and system prompts
- Voice activity detection shows "End of speech detected"

There's also a Python client implementation in `unmute/loadtest/loadtest_client.py` that demonstrates the protocol from a different angle - it's used for benchmarking.

### How the interruption by the user mid-reponse works

The interruption mechanism uses the **word-level timestamps** from DSM-TTS.

Here's how it works:

**During generation:**
- DSM-TTS outputs audio chunks along with precise timestamps for each word
- The system tracks exactly which words have been spoken and when

**When user interrupts:**
- The frontend detects the user starting to speak (via voice activity detection)
- It signals the backend to stop the current TTS generation
- Because the system knows the exact timestamp where it stopped, it knows which part of the LLM's response was actually spoken and which wasn't

**The clever part:**
The backend can then inform the LLM context about what was actually said vs. what was cut off. This means the conversation can continue naturally - the AI knows what the user heard and what they didn't.

The paper mentions: "If you interrupt mid-way through an explanation to ask a follow-up question, Unmute will know exactly where it got interrupted and which part of the explanation still remains to be said later."

### How the voice activity detection works

The Voice Activity Detection (VAD) in Unmute is particularly clever - it's **semantic** rather than just acoustic.

Traditional VAD Problem

Most voice systems use a separate VAD model that detects if someone is speaking or not, then waits a fixed time (like 500ms) after silence before deciding "they're done talking."

The problem: People naturally pause mid-sentence! A fixed timeout causes either:
- False positives (cutting people off mid-thought)
- Or long delays (waiting too long to be safe)

Kyutai's Semantic VAD Solution

Instead of a separate model, **DSM-ASR itself predicts the probability that the user is done talking**. It's built right into the speech-to-text model.

The key insight: The delay adapts based on **content and intonation**. The model can tell the difference between:
- "I went to the store..." (pause, more coming)
- "I went to the store." (done, falling intonation)

How It Works

The STT model outputs both:
1. Text transcription
2. End-of-speech probability

When this probability crosses a threshold, the system triggers the LLM response.

This is what you see in the UI when it shows "End of speech detected."

### Protocol and messages used between frontend and backend

**Message format:**
- Based on OpenAI Realtime API format
- Defined in `unmute/openai_realtime_api_events.py`
- Contains both standard ORA messages and custom Unmute extensions

**Audio handling:**
- Browser sends raw audio data over WebSocket
- Backend streams audio back for playback
- Real-time bidirectional communication

**Debug info:**
- Backend populates `self.debug_dict` in `unmute_handler.py`
- This gets sent to frontend for the dev mode view

WebSocket Connection

**Endpoint:** `/v1/realtime` using the `realtime` subprotocol
**Port:** 8000 (dev), or 80/443 through Traefik (production)

Audio Format

All audio uses:
- **Codec:** Opus
- **Sample rate:** 24kHz  
- **Channels:** Mono
- **Encoding:** Base64-encoded bytes

Key Message Types

**Client → Server:**

1. **`input_audio_buffer.append`** - Streams microphone audio to backend
2. **`session.update`** - Configures voice character and instructions (required to start!)

**Server → Client:**

1. **`response.audio.delta`** - Streams generated speech back
2. **`conversation.item.input_audio_transcription.delta`** - Real-time user transcription
3. **`response.text.delta`** - Text of what AI is saying (for display)
4. **`input_audio_buffer.speech_started/stopped`** - VAD events (currently ignored)
5. **`response.created`** - Signals assistant is generating a response
6. **`error`** - Error/warning messages

All messages are JSON with a common structure defined in `openai_realtime_api_events.py`.

The connection lifecycle starts with a health check at `/v1/health`, then establishes the WebSocket, sends session config, and starts bidirectional streaming.

### How the audio streaming works

Browser → Backend (Input Audio)

The browser continuously captures audio from your microphone and sends it in chunks:

1. Browser records audio in **Opus format** (24kHz, mono)
2. Encodes chunks as **base64**
3. Sends `input_audio_buffer.append` messages via WebSocket
4. Backend forwards this to the STT server
5. STT transcribes in real-time and detects when you stop speaking

Backend → Browser (Output Audio)

Once the LLM starts generating text:

1. Backend feeds text tokens to TTS as they arrive
2. TTS generates audio chunks (also Opus, 24kHz)
3. Backend sends `response.audio.delta` messages with base64 audio
4. Browser decodes and plays audio immediately
5. Continues until LLM finishes generating

Key Feature: Streaming in Text

The magic is that TTS doesn't wait for complete text - it starts generating audio from the first few tokens. Combined with the 220ms TTS latency, this creates very responsive conversations.

The browser also receives `response.text.delta` messages showing what's being said, which enables subtitles.

### Backend implementation

That's a broad question! The backend has quite a few components. Let me break down what I know, and you can tell me which parts you want to dive deeper into.

Backend Overview

**Main entry point:** `unmute/main_websocket.py` - handles WebSocket connections

**Key handler:** `unmute_handler.py` - orchestrates the conversation flow and populates `self.debug_dict` for debugging

Major Components

1. **WebSocket server** - receives audio, sends responses
2. **STT client** - connects to speech-to-text server via WebSocket
3. **LLM client** - queries text generation (supports any OpenAI-compatible API)
4. **TTS client** - connects to text-to-speech server via WebSocket
5. **Session management** - handles voice configs from `voices.yaml`
6. **System prompts** - defined in `unmute/llm/system_prompt.py`

Technology Stack

- **FastAPI** for the web server
- **WebSocket** for real-time communication
- **Python** with `uv` for dependency management

### Backend orchestration logic

The orchestration logic is the heart of how the backend coordinates everything. Here's what happens:

Main Flow

The backend acts as a **central coordinator** managing three concurrent streams:

**1. Audio Input Stream (You → STT)**
- Receives audio chunks from browser
- Forwards to STT server via WebSocket
- STT transcribes and watches for end-of-speech

**2. Text Generation (STT → LLM → TTS)**
- When STT detects you're done speaking
- Backend sends transcript to LLM
- LLM starts generating response tokens
- Tokens immediately stream to TTS (no waiting!)

**3. Audio Output Stream (TTS → You)**
- TTS generates audio from incoming text tokens
- Backend forwards audio chunks to browser
- Continues until LLM finishes

State Management

The backend must track:
- Current conversation state (listening vs. responding)
- Active sessions and their configurations
- Debug information in `self.debug_dict`
- Word timestamps for interruption handling

The "Flush Trick"

When end-of-speech is detected, the backend tells STT to process remaining audio at **4x speed** to reduce the 500ms delay to ~125ms.

### Concurrent connections

Batching Support

The key to handling concurrent connections is **batching** - remember how DSM operates at a constant framerate? This means:

- Multiple user conversations can be processed **simultaneously** on the same GPU
- All streams advance in lockstep (one frame per step)
- No need for complex scheduling or per-user logic

Throughput Numbers

From the documentation:

**STT (Rust server on L40S GPU):**
- 64 simultaneous connections at 3x real-time factor
- On H100: up to 400 concurrent streams!

**TTS (Rust server on L40S GPU):**
- 16 simultaneous connections at 2x real-time factor
- 32 connections with batching at 350ms latency

Implementation

The actual STT and TTS servers are **Rust-based** for performance, providing WebSocket endpoints that the Python backend connects to. The backend itself uses **FastAPI with WebSocket support** to handle multiple browser connections.

Each browser connection gets its own handler instance that manages that conversation's state independently.

### Rust servers

The Rust servers handle the actual STT and TTS inference. Here's what I know:

Purpose

The STT and TTS models run in **separate Rust servers** for performance reasons. They expose WebSocket endpoints that the Python backend connects to.

Performance Benefits

Rust provides:
- Lower latency than Python
- Better memory management
- Efficient GPU utilization for batching

Key Features

**STT Rust Server:**
- Processes audio faster than real-time (4x speed for flush trick)
- Handles batching of multiple streams
- Implements the semantic VAD

**TTS Rust Server:**
- Streams audio generation
- Handles the action stream for word timing
- Manages speaker embeddings

Repository

Both are in the `delayed-streams-modeling` GitHub repo (mentioned in the docs). The Unmute repo references this for running the servers.

The servers use WebSocket for communication, accepting audio/text input and streaming results back.

Multi-GPU Setup

The system can distribute components across different GPUs:

**Single GPU (16GB minimum):**
- All three components (STT, TTS, LLM) share one GPU
- Works but has higher latency (~750ms for TTS on L40S)

**Three GPU setup (recommended):**
- STT on GPU 0
- TTS on GPU 1  
- LLM on GPU 2
- Reduces TTS latency to ~450ms

Configuration

In Docker Compose, you can assign specific GPUs using:
```yaml
deploy:
  resources:
    reservations:
      devices:
        - driver: nvidia
          count: 1
          capabilities: [gpu]
```

Why This Helps

Each component runs independently without competing for GPU memory or compute. The Rust servers handle their own GPU batching efficiently.

On unmute.sh, they use this three-GPU setup for optimal performance.

### Docker compose file

#### Overview

This file defines 5 services that work together to run Unmute:
1. **traefik** - Reverse proxy/router
2. **frontend** - Next.js web interface
3. **backend** - Python orchestration service
4. **tts** - Text-to-speech Rust server
5. **stt** - Speech-to-text Rust server
6. **llm** - Language model (vLLM)

#### Traefik (Reverse Proxy)

This routes incoming HTTP requests to the right service:
- Listens on port 80
- Routes `/api/*` requests → backend
- Routes everything else → frontend
- Currently HTTP only (no HTTPS)

The priority system ensures API calls go to backend first (priority 100) before falling through to frontend (priority 10).

#### Frontend Service

**What it does:**
- Builds the Next.js frontend from the `frontend/` directory
- Uses a special hot-reloading Dockerfile for development
- Mounts the source code so changes appear instantly without rebuilding

**Traefik routing:**
- Catches all requests that don't match other routes
- Internally runs on port 3000
- Lowest priority (so backend API routes take precedence)

The volume mounting means you can edit frontend code and see changes immediately without restarting the container.

#### Backend Service

**What it does:**
- Builds from the root directory (contains Python code)
- Uses hot-reloading for development
- Mounts the `unmute/` directory for live code changes

**Environment variables:**
```yaml
environment:
  - KYUTAI_STT_URL=ws://stt:8080
  - KYUTAI_TTS_URL=ws://tts:8080
  - KYUTAI_LLM_URL=http://llm:8000
```

These tell the backend how to connect to the other services. Notice Docker's internal networking - `stt`, `tts`, and `llm` are service names that resolve automatically.

**Traefik routing:**

Requests to `/api/something` get routed here, and the `/api` prefix is stripped before reaching the backend (so it sees `/something`).

#### STT and TTS Services

Both have very similar configurations:

**Key points:**
- Both use the same Rust-based `moshi-server` image
- Different config files specify STT vs TTS behavior
- Need HuggingFace token to download models

**GPU access:**
```yaml
deploy:
  resources:
    reservations:
      devices:
        - driver: nvidia
          count: all
```

Currently set to use **all GPUs**. For multi-GPU setups, you'd change `count: all` to `count: 1` to dedicate one GPU per service.

#### LLM Service

```yaml
llm:
  image: vllm/vllm-openai:v0.9.1
  command:
    [
      "--model=meta-llama/Llama-3.2-1B-Instruct",
      "--max-model-len=1536",
      "--dtype=bfloat16",
      "--gpu-memory-utilization=0.4",
    ]
```

**What it does:**
- Runs vLLM (fast LLM inference server)
- Uses Llama 3.2 1B by default (small, fits in 16GB GPU)
- Exposes OpenAI-compatible API on port 8000

**Key parameters:**

`--max-model-len=1536` - Maximum conversation length (tokens). Higher = longer conversations but more memory.

`--gpu-memory-utilization=0.4` - Uses 40% of GPU memory. You can increase this if running LLM on dedicated GPU.

`--dtype=bfloat16` - Uses 16-bit precision for efficiency

**Volumes:**
Same caching strategy as STT/TTS to avoid re-downloading models.

**NOTE comments** in the file suggest places you might customize (different model, more memory, etc.).